In [1]:
import pddlpy

In [3]:
domain = ".\Problems\Groupe3\maze.pddl"
problem = '.\Problems\Groupe3\problems\maze_p0.pddl'

dp = pddlpy.DomainProblem(domain, problem)

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
C:\Users\link3\AppData\Local\Temp\ipykernel_9552\1175359565.py:1: SyntaxWarning: invalid escape sequence '\P'
  domain = ".\Problems\Groupe3\maze.pddl"
C:\Users\link3\AppData\Local\Temp\ipykernel_9552\1175359565.py:2: SyntaxWarning: invalid escape sequence '\P'
  problem = '.\Problems\Groupe3\problems\maze_p0.pddl'


In [22]:
def info_dp(dp):
    #print("Domain: " + str(dp.domain))
    print("Goals: " + str(dp.goals()))
    print("Ground operator: " + str(dp.ground_operator("move-down")))
    print("Initial state: " + str(dp.initialstate()))
    print("Operators: " + str(dp.operators()))
    #print("Problem: " + str(dp.problem))
    #print("Vargroundspace: " + str(dp.vargroundspace))
    print("World objects: " + str(dp.worldobjects()))

In [23]:
info_dp(dp)

Goals: {('at', 'maxime', 'x4', 'y4')}
Ground operator: <generator object DomainProblem.ground_operator at 0x0000025109251D80>
Initial state: {('dec', 'x2', 'x1'), ('inc', 'x3', 'x4'), ('dec', 'y4', 'y3'), ('at', 'maxime', 'x1', 'y1'), ('inc', 'y1', 'y2'), ('inc', 'y3', 'y4'), ('can_jump', 'maxime'), ('dec', 'x3', 'x2'), ('dec', 'y3', 'y2'), ('dec', 'y2', 'y1'), ('inc', 'x2', 'x3'), ('inc', 'y2', 'y3'), ('dec', 'x4', 'x3'), ('inc', 'x1', 'x2')}
Operators: dict_keys(['move-up', 'move-down', 'move-left', 'move-right', 'jump-up', 'jump-down', 'jump-left', 'jump-right'])
World objects: {'x4': 'position', 'x2': 'position', 'y4': 'position', 'maxime': 'agent', 'x1': 'position', 'y3': 'position', 'y2': 'position', 'x3': 'position', 'y1': 'position'}


On veut un plan

On utilise GraphPlan

On crée un graphe et on cherche un plan

On crée un graphe : forward
On cherche un plan : backward

Quand on fait le forward, à chaque nouveau state on fait un backward

Pour faire un forward :
- On prend un state
- On applique toutes les actions qu'on peut
- On crée un nouveau state avec les nouveaux prédicats
- On donne les nouveaux mutex

Appliquer une action :
- Regarder si ses préconditions sont respectées
- Donner les prédicats que cette action génère

Problème actuel :
Si on vérifie qu'il n'y a pas de murs, comment on fait si on n'a pas ce (non) prédicat dans notre state ?
- On regarde si le prédicat est dans la liste neg
	- Si oui, on peut appliquer l'action
	- Si non, on regarde si le prédicat est dans la liste pos
		- Si oui, on ne peut pas appliquer l'action
		- Si non, on peut appliquer l'action et on rajoute le prédicat dans la liste neg

In [238]:
# Notre state a une partie pos et une partie neg
class State:
	def __init__(self, pos: set, neg: set):
		self.pos = pos
		self.neg = neg

	def __str__(self):
		return f"pos : {self.pos}\nneg : {self.neg}"

	def copy(self):
		return State(self.pos.copy(), self.neg.copy())

In [265]:
class Action:
    def __init__(self, pre_pos, pre_neg, eff_pos, eff_neg):
        self.pre_pos = pre_pos
        self.pre_neg = pre_neg
        self.eff_pos = eff_pos
        self.eff_neg = eff_neg

    def __str__(self):
        return f"pre_pos : {self.pre_pos}\npre_neg : {self.pre_neg}\neff_pos : {self.eff_pos}\neff_neg : {self.eff_neg}\n"

    def __repr__(self):
        return f"pre_pos : {self.pre_pos}\npre_neg : {self.pre_neg}\neff_pos : {self.eff_pos}\neff_neg : {self.eff_neg}\n"

In [222]:
def set_str(s: set):
    s_str = set()
    for i in s:
        s_str.add(str(i))
    return s_str

In [223]:
def is_applicable(action, state):
    a_pos = set_str(action.precondition_pos)
    a_neg = set_str(action.precondition_neg)
    if not a_pos <= state.pos:
        return False
    if a_neg <= state.neg:
        return True
    else:
        if (a_neg - state.neg).isdisjoint(state.pos):
            state.neg.update(a_neg) # On choisit d'update le state plutôt que le next_state, la maj effectuée est sur des prédicats déjà présents (puisque hypothèse du monde clos)
            return True
        else:
            return False

In [224]:
def forward(actions, state):
    next_action = set()
    effect_pos = set()
    effect_neg = set()
    for action in actions:
        if is_applicable(action, state):
            p_pos = set_str(action.precondition_pos)
            p_neg = set_str(action.precondition_neg)
            e_pos = set_str(action.effect_pos)
            e_neg = set_str(action.effect_neg)
            next_action.add(Action(p_pos, p_neg, e_pos, e_neg))
            effect_pos.update(e_pos)
            effect_neg.update(e_neg)
    next_state = state.copy()
    next_state.pos.update(effect_pos)
    next_state.neg.update(effect_neg)
    return next_state, next_action

In [225]:
init = State(set_str(dp.initialstate()), set())

In [226]:
print(init)

pos : {"('dec', 'x2', 'x1')", "('dec', 'y3', 'y2')", "('inc', 'x1', 'x2')", "('dec', 'x3', 'x2')", "('dec', 'y2', 'y1')", "('dec', 'x4', 'x3')", "('at', 'maxime', 'x1', 'y1')", "('inc', 'y3', 'y4')", "('dec', 'y4', 'y3')", "('inc', 'y2', 'y3')", "('inc', 'x2', 'x3')", "('can_jump', 'maxime')", "('inc', 'y1', 'y2')", "('inc', 'x3', 'x4')"}
neg : set()


In [266]:
next_state = State(set(), set())
next_action = set()
for op in dp.operators():
    n_s, n_a = forward(dp.ground_operator(op), init)
    next_state.pos.update(n_s.pos)
    next_state.neg.update(n_s.neg)
    next_action.update(n_a)

In [263]:
print(next_state)

pos : {"('dec', 'x2', 'x1')", "('dec', 'y3', 'y2')", "('dec', 'x3', 'x2')", "('dec', 'y2', 'y1')", "('inc', 'x2', 'x3')", "('at', 'maxime', 'x1', 'y1')", "('inc', 'y3', 'y4')", "('dec', 'y4', 'y3')", "('inc', 'y2', 'y3')", "('at', 'maxime', 'x2', 'y1')", "('can_jump', 'maxime')", "('at', 'maxime', 'x1', 'y2')", "('inc', 'x1', 'x2')", "('dec', 'x4', 'x3')", "('inc', 'y1', 'y2')", "('inc', 'x3', 'x4')"}
neg : {"('at', 'maxime', 'x1', 'y1')", "('wall', 'x2', 'y1')", "('wall', 'x1', 'y2')"}


In [267]:
print(next_action)

{pre_pos : {"('inc', 'x1', 'x2')", "('at', 'maxime', 'x1', 'y1')"}
pre_neg : {"('wall', 'x2', 'y1')"}
eff_pos : {"('at', 'maxime', 'x2', 'y1')"}
eff_neg : {"('at', 'maxime', 'x1', 'y1')"}
, pre_pos : {"('at', 'maxime', 'x1', 'y1')", "('inc', 'y1', 'y2')"}
pre_neg : {"('wall', 'x1', 'y2')"}
eff_pos : {"('at', 'maxime', 'x1', 'y2')"}
eff_neg : {"('at', 'maxime', 'x1', 'y1')"}
}


In [260]:
print(list(next_action)[0])

pre_pos : {"('at', 'maxime', 'x1', 'y1')", "('inc', 'y1', 'y2')"}
pre_neg : {"('wall', 'x1', 'y2')"}
eff_pos : {"('at', 'maxime', 'x1', 'y2')"}
eff_neg : {"('at', 'maxime', 'x1', 'y1')"}
